# Chaining search


## Corpus search

* Run the cell below to show the UI, and fill in your search query

In [1]:
from chaininglib import ui

# Create corpus UI, creates references to field contents
corpusQueryField, corpusField = ui.create_corpus_ui()

 * Run the cell below to search

In [6]:
from chaininglib import search

query= corpusQueryField.value
corpus = corpusField.value
df_corpus = search.search_corpus(query,corpus)
display(df_corpus)



,left context,word 0,right context
0,of Imam Zij staat te,boek,als het eerste zwarte topmodel
1,microhandel in het land gestimuleerd,Boeken,met thema's die betrekking hebben
2,thema's Surinaamse opinie tijdschriften en,boeken,over winticultuur behoren ook tot
3,mee voorop lopen In zijn,boek,Verloren in wanorde geeft Karel
4,onafhankelijk forensische onderzoek naar de,boeken,te starten Dit moet een
5,al eerder gepubliceerd een nieuw,boek,van Orlando Emanuels is haast
6,104 bladzijden geen erg dik,boek,geworden maar het ziet er
7,vaak hoe je een goed,boek,kan onderscheiden Ik legde dan
8,gekscherend aan toe Een goed,boek,loopt slecht af En slecht
9,heb vaker met hem gesproken,boeken,aangehaald tijdschriften etc. etc. maar


## Lexicon search

* Run the cell below to show the UI, and fill in your search query in the UI

In [3]:
from chaininglib import ui
searchWordField, lexiconField = ui.create_lexicon_ui()

### Search!
 * Run the cell below to search

In [5]:
from chaininglib import queries, search

search_word = searchWordField.value
lexicon = lexiconField.value
# USER: can replace this by own custom query
query = queries.get_query(word=search_word, lexicon=lexicon)

df_lexicon = search.search_lexicon(query, lexicon)
df_lexicon_relevant = df_lexicon[["inputMode", "n_ontolex_writtenRep", "n_syndef_definitionText", "n_sensedef_definitionText", "wy_f_show", "wy_t_show"]]
display(df_lexicon_relevant)

,inputMode,n_ontolex_writtenRep,n_syndef_definitionText,n_sensedef_definitionText,wy_f_show,wy_t_show
0,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
1,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
2,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
3,lemma,boek,acte,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
4,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
5,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1228,1349
6,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
7,lemma,boek,oorkonde,"Ook in den zin van officieel stuk, acte, oorko...",1456,1456
8,lemma,boek,Boek,Boek.,1460,1480
9,lemma,boek,Boek,Boek.,1460,1480


## Case study: Frequency of *puur*+verb and *zuiver*+verb compared
* Below cell searches for *puur*+verb and for *zuiver*+verb in the CHN corpus
* Compare frequencies

In [15]:
from chaininglib import search
from IPython.core.display import display, HTML

# Word 1: puur
query1= r'[word="puur"][pos="verb"]'
corpus1 = "chn"
df_corpus1 = search.search_corpus(query1,corpus1)
display(HTML('<b>puur</b>'))
display(df_corpus1)

# Word 2: zuiver
query2= r'[word="zuiver"][pos="verb"]'
corpus2 = "chn"
df_corpus2 = search.search_corpus(query2,corpus2)
display(HTML('<b>zuiver</b>'))
display(df_corpus2)

# Compute difference
set_df1 = set(df_corpus1["word 1"])
set_df2 = set(df_corpus2["word 1"])
# Elements of 1 that are not in 2
diff_left = set_df1.difference(set_df2)
display(HTML('Werkwoorden voor <b>puur</b> niet in <b>zuiver</b>: ' + ", ".join(diff_left)))
# Elements of 2 that are not in 1
diff_right = set_df2.difference(set_df1)
display(HTML('Werkwoorden voor <b>zuiver</b> niet in <b>puur</b>: ' + ", ".join(diff_right)))

,left context,word 0,word 1,right context
0,stad omdat de cultuur daar,puur,is,
1,succes dan wanneer de coalitie,puur,gestoeld,is op een parlementaire meerderheid
2,de andere jongeren die gewoon,puur,willen,werken Maar financieel zijn zij
3,gratis zijn De deelname is,puur,gebaseerd,op interesse In de middaguren
4,de natuur De natuur is,puur,vertelt,de kunstenaar Een tijger jaagt
5,van een licentieovereenkomst maar het,puur,gaat,om de oorspronkelijke eis van
6,gevecht voor ons voorouderlijk land,puur,bedoeld,is om het menselijk leven
7,het SZF maar waar het,puur,gaat,om winsten te maken zegt
8,redactrice Inge SchelstraeteHET zou poëzie,puur,worden,waarschuwde Piet Piryns ons Maar
9,bepaalde zender te adverteren is,puur,gebaseerd,op marketing én op het


,left context,word 0,word 1,right context
0,baby Ik wil het contact,zuiver,beperken,tot de baby en het
1,zo zul je als mens,zuiver,moeten,zijn om in aanmerking te
2,adviezen van mensen Ik heb,zuiver,vastgelegd,wat ik zelf spraakmakend vond
3,karaoke je ding Daarvoor is,zuiver,zingen,niet nodig Dat is juist
4,aan als het gevoel erachter,zuiver,is,Met deze woorden besloot Loes
5,Maar als je je gevoelens,zuiver,houdt,en goede wensen creëert dan
6,Cronie maar die was te,zuiver,genomen,Bij The Scorpions benutten Misiedjan
7,Woudman op Cairo Deze werd,zuiver,genomen,door Dwight Tempico 2-1 Een
8,de leerlingen herkenbaar en is,zuiver,beweert,de directeur Ook enkele leerkrachten
9,bakzeil De strafschop was te,zuiver,genomen,
